In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

MEAN = 0.1307
STANDARD_DEVIATION = 0.3081

In [85]:
# Create model
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.drop1 = nn.Dropout2d(0.25)
    self.drop2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(5*5*64, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    # x = x.reshape(84, 84, 4)
    # x = torch.narrow(x, dim=2, start=3, length=1)
    # x = x.reshape(1, 1, 84, 84)
    # x = F.avg_pool2d(x, 3, stride=3)
    # x = x/255
    # x = (x - MEAN) / STANDARD_DEVIATION

    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.drop1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.drop2(x)
    x = self.fc2(x)

    # output = F.softmax(x)
    return x

In [86]:
model = Net()

In [88]:
model.load_state_dict(torch.load('drive/MyDrive/pytorch/practice/mnist_model2.pt'))

<All keys matched successfully>

In [89]:
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout2d(p=0.25, inplace=False)
  (drop2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [69]:
dummy_input = torch.zeros(1, 1, 28, 28)
torch.onnx.export(model, dummy_input, 'drive/MyDrive/pytorch/practice/ttmnist.onnx', verbose=True)

graph(%input.1 : Float(1:784, 1:784, 28:28, 28:1, requires_grad=0, device=cpu),
      %conv1.weight : Float(32:9, 1:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %conv1.bias : Float(32:1, requires_grad=1, device=cpu),
      %conv2.weight : Float(64:288, 32:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %conv2.bias : Float(64:1, requires_grad=1, device=cpu),
      %fc1.weight : Float(128:1600, 1600:1, requires_grad=1, device=cpu),
      %fc1.bias : Float(128:1, requires_grad=1, device=cpu),
      %fc2.weight : Float(10:128, 128:1, requires_grad=1, device=cpu),
      %fc2.bias : Float(10:1, requires_grad=1, device=cpu)):
  %9 : Float(1:21632, 32:676, 26:26, 26:1, requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1, 1]](%input.1, %conv1.weight, %conv1.bias) # /usr/local/lib/python3.6/dist-packages/torch/nn/modules/conv.py:420:0
  %10 : Float(1:21632, 32:676, 26:26, 26:1, requires_grad=1, device=cpu) = onnx::Relu(%9) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:
!pip install pytorch2keras

     |████████████████████████████████| 14.5MB 243kB/s 
  Created wheel for pytorch2keras: filename=pytorch2keras-0.2.4-cp36-none-any.whl size=29663 sha256=9c3b96470fee0a55aba75b7f18c423039e42b2c94b87cbe7d2019378e6407f5f
  Stored in directory: /root/.cache/pip/wheels/36/8b/2e/e2b6ae7a78ad4661e6156700bf96816309013f89f7a21f82d7
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-cp36-none-any.whl size=24580 sha256=5d00a9ef3ee1639ab57b466816c4c3573b6c213b64c4a4606cd60de212bf07ed
  Stored in directory: /root/.cache/pip/wheels/1d/d2/0a/8310ecf5f80355546fadda0fb4c611c8b54a63fce9af555155
Successfully built pytorch2keras onnx2keras


In [90]:
from pytorch2keras.converter import pytorch_to_keras
import numpy as np
from torch.autograd import Variable

In [91]:
input_np = np.random.uniform(0, 1, (1, 1, 28, 28))
input_var = Variable(torch.FloatTensor(input_np))

In [92]:
k_model = pytorch_to_keras(model, input_var)  

Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.
Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.


In [93]:
import tensorflow as tf

In [94]:
k_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
9 (Conv2D)                   (None, 32, 26, 26)        320       
_________________________________________________________________
10 (Activation)              (None, 32, 26, 26)        0         
_________________________________________________________________
11 (MaxPooling2D)            (None, 32, 13, 13)        0         
_________________________________________________________________
12 (Conv2D)                  (None, 64, 11, 11)        18496     
_________________________________________________________________
13 (Activation)              (None, 64, 11, 11)        0         
_________________________________________________________________
14 (MaxPooling2D)            (None, 64, 5, 5)          0   

In [38]:
# tf.saved_model.save(k_model, 'drive/MyDrive/pytorch/practice/k_model')
k_model.save('drive/MyDrive/pytorch/practice/k_model.h5')

INFO:tensorflow:Assets written to: drive/MyDrive/pytorch/practice/k_model/assets


In [15]:
# convert keras model to tensorflow.js format
!pip install tensorflowjs

     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 112kB 22.4MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [32]:
import tensorflowjs as tfjs

In [33]:
tfjs.converters.save_keras_model(k_model, 'drive/MyDrive/pytorch/practice/tfjsModel')

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
